# Inference for flu data in season 2019

## Flu case incidences in season 2019

In [ ]:
import numpy as np

import seirmo as se

flu_data = se.DatasetLibrary().french_flu()

In [ ]:
# Plot incidences data
plot = se.IncidenceNumberPlot()
plot.add_data(flu_data,)